<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/master/Debiasing_WE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Debiasing WE with CN

# Set up debiasing tool

In [1]:
# Setup:
# Clone the code repository from https://github.com/tolga-b/debiaswe.git
!mkdir debiaswe_tutorial
%cd debiaswe_tutorial
!git clone https://github.com/tolga-b/debiaswe.git


/content/debiaswe_tutorial
Cloning into 'debiaswe'...
remote: Enumerating objects: 91, done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 91
Unpacking objects: 100% (91/91), done.


In [2]:
#Download word embeddings provided by the author
!pip install gdown
!gdown https://drive.google.com/uc?id=1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG

Downloading...
From: https://drive.google.com/uc?id=1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG
To: /content/debiaswe_tutorial/w2v_gnews_small.zip
29.1MB [00:00, 79.8MB/s]


In [3]:
!unzip /content/debiaswe_tutorial/w2v_gnews_small.zip

Archive:  /content/debiaswe_tutorial/w2v_gnews_small.zip
  inflating: w2v_gnews_small.txt     


In [11]:
import numpy as np
import scipy, requests, codecs, os, re, nltk, itertools, csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering, KMeans
import tensorflow as tf
from scipy.stats import spearmanr
import pandas as pd
import functools as ft
import os
import io
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np
from debiaswe import debiaswe
from debiaswe.debiaswe import we
import debiaswe as dwe
#import debiaswe.we as we
from debiaswe.debiaswe.we import WordEmbedding
from debiaswe.debiaswe.data import load_professions

# Load dataset

In [5]:
%cd /content/debiaswe_tutorial/debiaswe

/content/debiaswe_tutorial/debiaswe


In [6]:
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f) #gender definitional words
print("definitional", defs)
defs_list = []
for pair in defs:
  defs_list.append(pair[0])
  defs_list.append(pair[1])

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f) 
print(equalize_pairs)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['woman', 'man'], ['girl', 'boy'], ['she', 'he'], ['mother', 'father'], ['daughter', 'son'], ['gal', 'guy'], ['female', 'male'], ['her', 'his'], ['herself', 'himself'], ['Mary', 'John']]
[['monastery', 'convent'], ['spokesman', 'spokeswoman'], ['Catholic_priest', 'nun'], ['Dad', 'Mom'], ['Men', 'Women'], ['councilman', 'councilwoman'], ['grandpa', 'grandma'], ['grandsons', 'granddaughters'], ['prostate_cancer', 'ovarian_cancer'], ['testosterone', 'estrogen'], ['uncle', 'aunt'], ['wives', 'husbands'], ['Father', 'Mother'], ['Grandpa', 'Grandma'], ['He', 'She'], ['boy', 'girl'], ['boys', 'girls'], ['brother', 'sister'], ['brothers', 'sisters'], ['businessman', 'businesswoman'], ['chairman', 'chairwoman'], ['colt', 'filly'], ['congressman', 'congresswoman'], ['dad', 'mom'], ['dads', 'moms'], ['dudes', 'gals'], ['ex_girlfriend', 'ex_boyfriend'], ['father', 'mother'], ['fatherhood', 'motherhood'], ['fathers', 'mothers'], ['fella', 'granny'], ['fraternity', 'sorority'], ['geldi

In [0]:
import gensim
from gensim.models.keyedvectors import KeyedVectors

In [8]:
!python -m gensim.scripts.glove2word2vec -i /content/debiaswe_tutorial/w2v_gnews_small.txt -o /content/small_w2v.txt

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'gensim.scripts.glove2word2vec' found in sys.modules after import of package 'gensim.scripts', but prior to execution of 'gensim.scripts.glove2word2vec'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-02-14 22:31:19,961 - glove2word2vec - INFO - running /usr/local/lib/python3.6/dist-packages/gensim/scripts/glove2word2vec.py -i /content/debiaswe_tutorial/w2v_gnews_small.txt -o /content/small_w2v.txt
2019-02-14 22:31:20,024 - glove2word2vec - INFO - converting 26423 vectors from /content/debiaswe_tutorial/w2v_gnews_small.txt to /content/small_w2v.txt
2019-02-14 22:31:20,269 - glove2word2vec - INFO - Converted model with 26423 vectors and 300 dimensions


In [25]:
#load word2vec file in a correct form
w2v = KeyedVectors.load_word2vec_format('/content/small_w2v.txt', binary=False)
print('The w2v embedding has been loaded!')

The w2v embedding has been loaded!


In [12]:
nltk.download('punkt')
from numpy.linalg import norm, inv, eig

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def protoConceptor(wordVecModel_str, defs_list, alpha):
    # compute the prototype conceptor with alpha = 2
    
    
    wordVecModel = eval(wordVecModel_str)    



    # put the word vectors in columns
    x_collector = wordVecModel.vectors[:].T   #total embeddings    
        
    num_vec = x_collector.shape[0]
    cn_vec= []
    for word in defs_list:
      if word in wordVecModel.vocab:
        cn_vec.append(wordVecModel[word])
    cn_collector = np.transpose(cn_vec) # embeddings of gender specific words
    nrWords = cn_collector.shape[1] # number of gender specific words
    
    
    R = cn_collector.dot(cn_collector.T) / nrWords # calculate the correlation matrix
    
    C = R @ inv(R + alpha ** (-2) * np.eye(300))# calculate the conceptor matrix
    mat = ((np.eye(num_vec)-C)@ x_collector).T #transform total embeddings
    return  mat

In [0]:
cn_f_name = 'w2v'
w2v_cn_mat = protoConceptor(cn_f_name, defs_list, 2)

In [0]:
#write CN word vecs into file
model = eval('w2v')
f = open('/content/w2v_cn.txt','w+')
for word in model.vocab:
  f.write(word+' ')
  id = model.vocab[word].index
  for item in w2v_cn_mat[id,:]:
    f.write(str(item)+' ')
  f.write('\n')
f.close()

In [18]:
!python -m gensim.scripts.glove2word2vec -i /content/w2v_cn.txt -o /content/w2v_cn_2.txt

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'gensim.scripts.glove2word2vec' found in sys.modules after import of package 'gensim.scripts', but prior to execution of 'gensim.scripts.glove2word2vec'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-02-14 22:34:42,270 - glove2word2vec - INFO - running /usr/local/lib/python3.6/dist-packages/gensim/scripts/glove2word2vec.py -i /content/w2v_cn.txt -o /content/w2v_cn_2.txt
2019-02-14 22:34:42,412 - glove2word2vec - INFO - converting 26423 vectors from /content/w2v_cn.txt to /content/w2v_cn_2.txt
2019-02-14 22:34:42,973 - glove2word2vec - INFO - Converted model with 26423 vectors and 300 dimensions


In [0]:
cn_w2v = KeyedVectors.load_word2vec_format('/content/w2v_cn_2.txt', binary=False)

In [20]:
# load professions words
professions = load_professions()
profession_words = [p[0] for p in professions]

Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


# Direct Bias Results

# TOP 20 neighbors on he-she axis 

After CN, professions words are closer to the middle on he-she axis.

In [26]:
# load google news word2vec
E = WordEmbedding('/content/debiaswe_tutorial/w2v_gnews_small.txt')
# gender direction
v_gender = E.diff('she', 'he')

*** Reading data from /content/debiaswe_tutorial/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine


In [27]:
#sp projected words onto he-she axis and sorted neighbors based on distance
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

#Top 20 neigbors of he and top 20 neighbors of she, and their distances to he/she
sp[0:20],sp[-20:]

([(-0.23798442, 'maestro'),
  (-0.21665451, 'statesman'),
  (-0.20758669, 'skipper'),
  (-0.20267202, 'protege'),
  (-0.2020676, 'businessman'),
  (-0.19492392, 'sportsman'),
  (-0.18836352, 'philosopher'),
  (-0.1807366, 'marksman'),
  (-0.1728986, 'captain'),
  (-0.16785555, 'architect'),
  (-0.16702037, 'financier'),
  (-0.16313636, 'warrior'),
  (-0.15280862, 'major_leaguer'),
  (-0.15001445, 'trumpeter'),
  (-0.14718868, 'broadcaster'),
  (-0.14637242, 'magician'),
  (-0.14401694, 'fighter_pilot'),
  (-0.13782285, 'boss'),
  (-0.137182, 'industrialist'),
  (-0.13684885, 'pundit')],
 [(0.19714224, 'interior_designer'),
  (0.20833439, 'housekeeper'),
  (0.21560375, 'stylist'),
  (0.2236317, 'bookkeeper'),
  (0.23776126, 'maid'),
  (0.24125955, 'nun'),
  (0.24782579, 'nanny'),
  (0.24929334, 'hairdresser'),
  (0.24946158, 'paralegal'),
  (0.25276464, 'ballerina'),
  (0.25718823, 'socialite'),
  (0.26647124, 'librarian'),
  (0.27317622, 'receptionist'),
  (0.27540293, 'waitress'),
  (

In [28]:
#Load CN word embeddings
E_cn = WordEmbedding('/content/w2v_cn.txt')

*** Reading data from /content/w2v_cn.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine


In [0]:
v_gender_cn = E_cn.diff('she', 'he')

In [30]:
# Top 20 neighbors after CN
sp = sorted([(E_cn.v(w).dot(v_gender_cn), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.19818926, 'maestro'),
  (-0.18967961, 'statesman'),
  (-0.1880296, 'businessman'),
  (-0.18281998, 'skipper'),
  (-0.18042295, 'protege'),
  (-0.17646421, 'sportsman'),
  (-0.16928828, 'captain'),
  (-0.16235389, 'warrior'),
  (-0.16157728, 'philosopher'),
  (-0.15046968, 'marksman'),
  (-0.13698363, 'major_leaguer'),
  (-0.1355408, 'fighter_pilot'),
  (-0.132551, 'financier'),
  (-0.13202478, 'lieutenant'),
  (-0.13006216, 'trumpeter'),
  (-0.13000073, 'industrialist'),
  (-0.12769677, 'architect'),
  (-0.12576684, 'broadcaster'),
  (-0.122493416, 'mechanic'),
  (-0.12220184, 'magician')],
 [(0.1333736, 'housekeeper'),
  (0.13594729, 'maid'),
  (0.14111671, 'interior_designer'),
  (0.15676282, 'nun'),
  (0.16862068, 'stylist'),
  (0.17241657, 'ballerina'),
  (0.17548509, 'socialite'),
  (0.17573729, 'nanny'),
  (0.1771008, 'nurse'),
  (0.18136254, 'bookkeeper'),
  (0.18577197, 'librarian'),
  (0.19130343, 'waitress'),
  (0.19306836, 'paralegal'),
  (0.1936686, 'hairdresser'),
  

# Indirect Bias results
After CN negation, words on softball-football axis became less gender biased.

In [0]:
v_gender = E.diff('softball', 'football')
v_gender_cn = E_cn.diff('softball', 'football')

In [32]:
#after debiasing, softball-football direction is no longer gender biased
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.3378573, 'footballer'),
  (-0.19598085, 'businessman'),
  (-0.19320746, 'pundit'),
  (-0.18045807, 'maestro'),
  (-0.16597804, 'cleric'),
  (-0.16489363, 'marksman'),
  (-0.1635845, 'hitman'),
  (-0.16318859, 'philosopher'),
  (-0.15652362, 'statesman'),
  (-0.15603065, 'sportsman'),
  (-0.15583786, 'broadcaster'),
  (-0.1511248, 'banker'),
  (-0.1446815, 'boss'),
  (-0.14366435, 'gangster'),
  (-0.14306568, 'midfielder'),
  (-0.13627477, 'financier'),
  (-0.13474312, 'policeman'),
  (-0.13387913, 'cameraman'),
  (-0.1336779, 'vice_chancellor'),
  (-0.13013485, 'architect')],
 [(0.07963051, 'stylist'),
  (0.080893494, 'housekeeper'),
  (0.0827892, 'actress'),
  (0.08613199, 'treasurer'),
  (0.08741376, 'businesswoman'),
  (0.088028535, 'counselor'),
  (0.09069986, 'librarian'),
  (0.09132401, 'nanny'),
  (0.09736764, 'publicist'),
  (0.100761496, 'therapist'),
  (0.101995654, 'soloist'),
  (0.114671476, 'infielder'),
  (0.116350956, 'valedictorian'),
  (0.13610226, 'clerk'),
  (0

In [33]:
sp = sorted([(E_cn.v(w).dot(v_gender_cn), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.34442955, 'footballer'),
  (-0.17447214, 'pundit'),
  (-0.16029751, 'cleric'),
  (-0.15683536, 'businessman'),
  (-0.14433442, 'midfielder'),
  (-0.14430456, 'hitman'),
  (-0.14382309, 'vice_chancellor'),
  (-0.1408222, 'banker'),
  (-0.1355176, 'maestro'),
  (-0.13379517, 'broadcaster'),
  (-0.13157055, 'lecturer'),
  (-0.13030419, 'marksman'),
  (-0.12713684, 'philosopher'),
  (-0.11896048, 'boss'),
  (-0.11675761, 'sportsman'),
  (-0.11546311, 'statesman'),
  (-0.11324482, 'gangster'),
  (-0.11297941, 'firebrand'),
  (-0.11291969, 'cameraman'),
  (-0.112491034, 'policeman')],
 [(0.07075992, 'attorney'),
  (0.07231749, 'nanny'),
  (0.07735392, 'patrolman'),
  (0.07913429, 'major_leaguer'),
  (0.079291396, 'counselor'),
  (0.08319657, 'treasurer'),
  (0.08672979, 'publicist'),
  (0.08748335, 'trooper'),
  (0.08990188, 'therapist'),
  (0.089979075, 'investigator'),
  (0.09167763, 'soloist'),
  (0.10601995, 'homemaker'),
  (0.11213362, 'valedictorian'),
  (0.1140725, 'paralegal'),

# Hard Debiasing

In [34]:
%cd /content/debiaswe_tutorial/debiaswe/debiaswe/

/content/debiaswe_tutorial/debiaswe/debiaswe


In [0]:
from debiaswe.debiaswe import debias

In [0]:
from debiaswe.debiaswe.debias import debias

In [37]:
%cd /content/debiaswe_tutorial/debiaswe

/content/debiaswe_tutorial/debiaswe


In [38]:
E = WordEmbedding('/content/debiaswe_tutorial/w2v_gnews_small.txt')

*** Reading data from /content/debiaswe_tutorial/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine


In [39]:
#Using authos's debiasing function
debias(E, gender_specific_words, defs, equalize_pairs)

26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
{('Spokesman', 'Spokeswoman'), ('dad', 'mom'), ('boys', 'girls'), ('Fraternity', 'Sorority'), ('Boys', 'Girls'), ('DAD', 'MOM'), ('grandpa', 'grandma'), ('Fathers', 'Mothers'), ('GRANDFATHER', 'GRANDMOTHER'), ('catholic_priest', 'nun'), ('Gentlemen', 'Ladies'), ('sons', 'daughters'), ('King', 'Queen'), ('MONASTERY', 'CONVENT'), ('Congressman', 'Congresswoman'), ('father', 'mother'), ('spokesman', 'spokeswoman'), ('males', 'females'), ('male', 'female'), ('HE', 'SHE'), ('man', 'woman'), ('grandsons', 'granddaughters'), ('FATHERS', 'MOTHERS'), ('TWIN_BROTHER', 'TWIN_SISTER'), ('testosterone', 'estrogen'), ('grandfather', 'grandmother'), ('Man', 'Woman'), ('Catholic_Priest', 'Nun'), ('Men', 'Women'), ('SONS', 'DAUGHTERS'), ('Grandpa', 'Grandma'), ('Dudes', 'Gals'), ('Testosterone', 'Estrogen'), ('kings', 'queens'), ('monastery', 'convent'), ('Fatherhood', 'Motherhood'), ('colt', 'filly'), ('gentleman', 'lady

In [40]:
%cd /content

/content


In [41]:
#Save debiased word embeddings
E.save('debiased.txt')

Wrote 26423 words to debiased.txt


In [42]:
E_de = WordEmbedding('/content/debiased.txt')

*** Reading data from /content/debiased.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine


#Direct Bias(Hard Debiasing)

In [0]:
v_gender_de =E_de.diff('she', 'he')

In [44]:
# profession analysis gender
sp_debiased = sorted([(E_de.v(w).dot(v_gender_de), w) for w in profession_words])

sp_debiased[0:20], sp_debiased[-20:]

([(-0.43325222, 'businessman'),
  (-0.42799947, 'congressman'),
  (-0.36525276, 'dad'),
  (-0.35791722, 'councilman'),
  (-0.22225758, 'statesman'),
  (-0.15711337, 'salesman'),
  (-0.106753685, 'handyman'),
  (-0.08172015, 'monk'),
  (-3.8995495e-08, 'soldier'),
  (-3.7778136e-08, 'policeman'),
  (-3.5121047e-08, 'teenager'),
  (-3.421883e-08, 'drug_addict'),
  (-3.4161076e-08, 'politician'),
  (-3.3038305e-08, 'confesses'),
  (-3.2116077e-08, 'swimmer'),
  (-3.1512172e-08, 'prisoner'),
  (-3.1272066e-08, 'aide'),
  (-3.1170202e-08, 'senator'),
  (-3.0962838e-08, 'athlete'),
  (-3.0879164e-08, 'parliamentarian')],
 [(-8.0035534e-11, 'artiste'),
  (-7.2759576e-11, 'pundit'),
  (2.6921043e-10, 'researcher'),
  (5.7480065e-10, 'undersecretary'),
  (6.1118044e-10, 'tycoon'),
  (1.0772965e-09, 'author'),
  (1.4551915e-09, 'dermatologist'),
  (1.4842954e-09, 'interior_designer'),
  (1.5188562e-09, 'saxophonist'),
  (3.8307917e-09, 'illustrator'),
  (4.4237822e-09, 'associate_professor'),
  

# Indirect Bias(Hard Debiasing)

In [0]:
v_gender_de =E_de.diff('softball', 'football')

In [46]:
sp_debiased = sorted([(E_de.v(w).dot(v_gender_de), w) for w in profession_words])

sp_debiased[0:20], sp_debiased[-20:]

([(-0.32501778, 'footballer'),
  (-0.16693358, 'cleric'),
  (-0.15964463, 'vice_chancellor'),
  (-0.15362918, 'lecturer'),
  (-0.1531749, 'midfielder'),
  (-0.1515184, 'fashion_designer'),
  (-0.14984937, 'housewife'),
  (-0.14895485, 'pundit'),
  (-0.13962688, 'student'),
  (-0.13837737, 'banker'),
  (-0.12876885, 'hitman'),
  (-0.126078, 'civil_servant'),
  (-0.12569053, 'chancellor'),
  (-0.124440275, 'socialite'),
  (-0.121996544, 'journalist'),
  (-0.11445205, 'artiste'),
  (-0.11225906, 'entertainer'),
  (-0.1106455, 'broadcaster'),
  (-0.11030317, 'serviceman'),
  (-0.10903629, 'filmmaker')],
 [(0.046729255, 'deputy'),
  (0.048755586, 'geologist'),
  (0.049729493, 'paralegal'),
  (0.050405726, 'councilman'),
  (0.05245539, 'lifeguard'),
  (0.05282015, 'organist'),
  (0.055579696, 'assassin'),
  (0.058301304, 'instructor'),
  (0.060144324, 'soloist'),
  (0.064725064, 'attorney'),
  (0.06975241, 'foreman'),
  (0.07452034, 'treasurer'),
  (0.07646255, 'trooper'),
  (0.0775902, 'val

# Compare debiasing and non-debiasing results

In [47]:
%cd /content/

/content


In [48]:
!wget https://raw.githubusercontent.com/liutianlin0121/Conceptor-Negation-WV/master/data/wordSimData/EN-MEN-TR-3k.txt
!wget https://raw.githubusercontent.com/liutianlin0121/Conceptor-Negation-WV/master/data/wordSimData/EN-MTurk-287.txt
!wget https://raw.githubusercontent.com/liutianlin0121/Conceptor-Negation-WV/master/data/wordSimData/EN-RG-65.txt
!wget https://raw.githubusercontent.com/liutianlin0121/Conceptor-Negation-WV/master/data/wordSimData/EN-RW-STANFORD.txt
!wget https://raw.githubusercontent.com/liutianlin0121/Conceptor-Negation-WV/master/data/wordSimData/EN-SIMLEX-999.txt
!wget https://raw.githubusercontent.com/liutianlin0121/Conceptor-Negation-WV/master/data/wordSimData/EN-SimVerb-3500.txt
!wget https://raw.githubusercontent.com/liutianlin0121/Conceptor-Negation-WV/master/data/wordSimData/EN-WS-353-ALL.txt
!ls

--2019-02-14 22:39:29--  https://raw.githubusercontent.com/liutianlin0121/Conceptor-Negation-WV/master/data/wordSimData/EN-MEN-TR-3k.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53593 (52K) [text/plain]
Saving to: ‘EN-MEN-TR-3k.txt’

EN-MEN-TR-3k.txt    100%[===================>]  52.34K  --.-KB/s    in 0.01s   

2019-02-14 22:39:29 (3.79 MB/s) - ‘EN-MEN-TR-3k.txt’ saved [53593/53593]

--2019-02-14 22:39:29--  https://raw.githubusercontent.com/liutianlin0121/Conceptor-Negation-WV/master/data/wordSimData/EN-MTurk-287.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, a

In [0]:
def get_sim_no_cn(data_f_name, f_name):
  model = eval(f_name)
  fin = io.open(data_f_name, 'r', encoding='utf-8', newline='\n', errors='ignore')
  data = []
  line_num = 0
  for line in fin:
      tokens = line.rstrip().split()
      if tokens[0] in model.vocab and tokens[1] in model.vocab:
        data.append(((tokens[0], tokens[1]), float(tokens[2])))
 
  data.sort(key = lambda score: -score[1]) #sort based on score
  dataset = {}
  dataset_list = []
  
  for ((word1, word2), score) in data:
    sim_score = 1 - cosine_similarity(model[word1].reshape(1,-1), model[word2].reshape(1,-1))
    dataset[(word1, word2)] = sim_score
    dataset_list.append(((word1, word2),sim_score))
  dataset_list.sort(key = lambda score: score[1])
  spearman_list1=[]
  spearman_list2=[]
  for pos_1, (pair, score_1) in enumerate(data):
    score_2 = dataset[pair]
    pos_2 = dataset_list.index((pair, score_2))
    spearman_list1.append(pos_1)
    spearman_list2.append(pos_2)
  rho = spearmanr(spearman_list1, spearman_list2)
  return rho[0] 

There is a slight difference between the results below and the results in paper. The reason is that the author said that their word embeddings dataset has 26377 words, but the dataset they provided on github has 26423 words. I tried data cleaning methods discussed in their paper but still got 26423 words. The difference in entries would cause the difference. 

In their paper, they only used EN-RG-65 and EN-WS-353.

In [61]:
dataSets = ['EN-RG-65.txt', 'EN-WS-353-ALL.txt', 'EN-RW-STANFORD.txt', 'EN-MEN-TR-3k.txt', 'EN-MTurk-287.txt', 'EN-SIMLEX-999.txt', 'EN-SimVerb-3500.txt']
for dataset in dataSets:
    dataSetAddress = '/content/'+  dataset
    print('evaluating the data set', dataSetAddress)
    print('w2v ')
    print("%.4f" % get_sim_no_cn(dataSetAddress, 'w2v'))

evaluating the data set /content/EN-RG-65.txt
w2v 
0.7621
evaluating the data set /content/EN-WS-353-ALL.txt
w2v 
0.6880
evaluating the data set /content/EN-RW-STANFORD.txt
w2v 
0.6542
evaluating the data set /content/EN-MEN-TR-3k.txt
w2v 
0.7742
evaluating the data set /content/EN-MTurk-287.txt
w2v 
0.6958
evaluating the data set /content/EN-SIMLEX-999.txt
w2v 
0.4448
evaluating the data set /content/EN-SimVerb-3500.txt
w2v 
0.3562


In [51]:
!python -m gensim.scripts.glove2word2vec -i /content/debiased.txt -o /content/debiased_w2v.txt

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'gensim.scripts.glove2word2vec' found in sys.modules after import of package 'gensim.scripts', but prior to execution of 'gensim.scripts.glove2word2vec'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-02-14 22:39:56,130 - glove2word2vec - INFO - running /usr/local/lib/python3.6/dist-packages/gensim/scripts/glove2word2vec.py -i /content/debiased.txt -o /content/debiased_w2v.txt
2019-02-14 22:39:56,208 - glove2word2vec - INFO - converting 26423 vectors from /content/debiased.txt to /content/debiased_w2v.txt
2019-02-14 22:39:56,490 - glove2word2vec - INFO - Converted model with 26423 vectors and 300 dimensions


In [0]:
w2v_de = KeyedVectors.load_word2vec_format('/content/debiased_w2v.txt', binary=False)

In [53]:
dataSets = ['EN-RG-65.txt', 'EN-WS-353-ALL.txt', 'EN-RW-STANFORD.txt', 'EN-MEN-TR-3k.txt', 'EN-MTurk-287.txt', 'EN-SIMLEX-999.txt', 'EN-SimVerb-3500.txt']
for dataset in dataSets:
    dataSetAddress = '/content/'+  dataset
    print('evaluating the data set', dataSetAddress)
    print('w2v ')
    print("%.4f" % get_sim_no_cn(dataSetAddress, 'w2v_de'))

evaluating the data set /content/EN-RG-65.txt
w2v 
0.7604
evaluating the data set /content/EN-WS-353-ALL.txt
w2v 
0.6850
evaluating the data set /content/EN-RW-STANFORD.txt
w2v 
0.6544
evaluating the data set /content/EN-MEN-TR-3k.txt
w2v 
0.7730
evaluating the data set /content/EN-MTurk-287.txt
w2v 
0.6957
evaluating the data set /content/EN-SIMLEX-999.txt
w2v 
0.4474
evaluating the data set /content/EN-SimVerb-3500.txt
w2v 
0.3562


# CN results

In [58]:
dataSets = ['EN-RG-65.txt', 'EN-WS-353-ALL.txt', 'EN-RW-STANFORD.txt', 'EN-MEN-TR-3k.txt', 'EN-MTurk-287.txt', 'EN-SIMLEX-999.txt', 'EN-SimVerb-3500.txt']
for dataset in dataSets:
    dataSetAddress = '/content/'+  dataset
    print('evaluating the data set', dataSetAddress)
    print('w2v ')
    print("%.4f" % get_sim_no_cn(dataSetAddress, 'cn_w2v'))

evaluating the data set /content/EN-RG-65.txt
w2v 
0.7628
evaluating the data set /content/EN-WS-353-ALL.txt
w2v 
0.6886
evaluating the data set /content/EN-RW-STANFORD.txt
w2v 
0.6568
evaluating the data set /content/EN-MEN-TR-3k.txt
w2v 
0.7765
evaluating the data set /content/EN-MTurk-287.txt
w2v 
0.6889
evaluating the data set /content/EN-SIMLEX-999.txt
w2v 
0.4584
evaluating the data set /content/EN-SimVerb-3500.txt
w2v 
0.3625


# Soft Debiasing(Not working)

In [0]:
!pip install cvxpy

In [0]:
import cvxpy as cp

In [0]:
w2v_mat = w2v.vectors

In [0]:
w2v_mat.shape

(26423, 300)

In [0]:
U,s,V = np.linalg.svd(w2v_mat)

In [0]:
V.shape

(300, 300)

In [0]:
P = []
word_list = []
for pair in equalize_pairs:
  if pair[0] in w2v.vocab:
    P.append(w2v[pair[0]])
    word_list.append(pair[0])
  if pair[1] in w2v.vocab:
    P.append(w2v[pair[1]])
    word_list.append(pair[1])
P = np.matrix(P)


In [0]:
P.shape

(92, 300)

In [0]:
B = []
B.append(w2v['she'])
B.append(w2v['he'])
B = np.matrix(B)

In [0]:
B.shape

(2, 300)

In [0]:
print(len(P))

92


In [0]:
mat1 = np.diag(s).dot(V)*((X-np.identity(300)))*(np.transpose(V).dot(np.diag(s)))
#mat2= mat1*((X-np.identity(50)))*(np.transpose(V).dot(np.diag(s)))
#.dot((X-np.identity(50))).dot(np.transpose(V).dot(np.diag(s)))

In [0]:
mat2 = P*(X)*(np.transpose(B))

In [0]:
mat1

Expression(AFFINE, UNKNOWN, (300, 300))

In [0]:
mat2 = P.dot(X).dot(np.transpose(B))

In [0]:
norm1 = np.linalg.norm(mat1)

In [0]:
X = cp.Variable((300, 300), PSD=True)

In [0]:
X-np.identity(50)

Expression(AFFINE, UNKNOWN, (50, 50))

In [0]:
norm1= cp.norm(X-np.identity(300))

In [0]:
obj = cp.Minimize(cp.norm(mat1)+cp.norm(mat2))

In [0]:
problem = cp.Problem(cp.Minimize(cp.norm(mat1)+cp.norm(mat2)))

In [0]:
problem.solve()

In [0]:
for variable in problem.variables():
  print(variable.value[0])

[ 9.99998337e-01 -6.57498877e-07 -5.97286388e-07  2.78589948e-07
  5.21181077e-08 -2.28734878e-07 -1.23440125e-07 -5.24961302e-07
  3.76656985e-07  1.02171139e-06 -6.75971329e-07  1.01996925e-06
 -1.00146025e-06 -3.93990747e-07 -9.84867768e-07 -3.37496205e-07
  1.18584360e-06  9.59675581e-07 -6.42748275e-07 -3.42649841e-07
 -2.71466851e-07  1.33458664e-07  3.39883574e-07  1.06921398e-06
 -3.35677338e-07 -7.19774741e-07 -9.07763611e-08 -4.74541765e-07
  7.59212244e-08 -3.90731435e-07  5.93059205e-07 -2.41158553e-07
 -4.85931116e-07 -8.76911984e-07  4.80233698e-07  3.99723598e-07
  1.54123588e-07  2.36008567e-08  6.02180300e-08  8.92711512e-07
  4.28578838e-08  1.93770961e-07 -2.75466254e-06  1.88568188e-06
  7.37080854e-07  2.24333292e-07  4.36322358e-07  3.75663522e-07
  1.19390824e-07  4.17649000e-07]


In [0]:
X = cp.Variable((300, 300), PSD=True)
obj = cp.Minimize(cp.norm(np.diag(s).dot(V).dot((X-np.identity(300))).dot(np.transpose(V).dot(np.diag(s)))) + cp.norm(P.dot(X).dot(np.transpose(B))))

NameError: ignored

In [0]:
prob = cp.Problem(obj)
prob.solve()

In [0]:
len(word)

26423

In [0]:
f = open('/content/soft_de.txt', 'w+')
word = list(w2v.vocab)
for i in range(len(word)):
  f.write(word[i]+' ')
  for j in soft_mat[i]:
    f.write(str(j)+' ')
  f.write('\n')
f.close()

In [0]:
!python -m gensim.scripts.glove2word2vec -i /content/soft_de.txt -o /content/soft_w2v.txt

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'gensim.scripts.glove2word2vec' found in sys.modules after import of package 'gensim.scripts', but prior to execution of 'gensim.scripts.glove2word2vec'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-02-13 16:34:49,271 - glove2word2vec - INFO - running /usr/local/lib/python3.6/dist-packages/gensim/scripts/glove2word2vec.py -i /content/soft_de.txt -o /content/soft_w2v.txt
2019-02-13 16:34:49,411 - glove2word2vec - INFO - converting 26423 vectors from /content/soft_de.txt to /content/soft_w2v.txt
2019-02-13 16:34:50,335 - glove2word2vec - INFO - Converted model with 26423 vectors and 300 dimensions


In [0]:
soft_w2v = KeyedVectors.load_word2vec_format('/content/soft_w2v.txt', binary=False)